In [ ]:
import numpy as np
import matplotlib.pylab as plt
import matplotlib.cbook
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AR
import sys
sys.path.append('../zdrojaky')
from tsplot import tsplot
import warnings
warnings.simplefilter(action='ignore', category=(FutureWarning,matplotlib.cbook.mplDeprecation))

# Konstrukce a odhad AR modelu

Máme k dispozici burzovní ceny akcií společnosti Google v období od 7. února 2005 do 7. července 2005. Ty jsou uloženy v souboru `googlestock.txt`, který vypadá následovně:
```
date	   price
2/7/2005    196.03
2/8/2005    198.64
2/9/2005    191.58
2/10/2005   187.98
2/11/2005   187.4
2/14/2005   192.99
```

Načtěte data z druhého sloupce do proměnné `data`, ideálně pro jednoduchost do numpy.array. Do proměnné `ndat` uložte počet měření a vykreslete řadu, ACF a PACF pomocí funkce `tsplot()`. Interpretujte, co grafy znázorňují.

In [ ]:
data = np.genfromtxt('googlestock.txt', ***)
ndat = data.size
tsplot(data)

Vykreslete 4 grafy - scatterploty - znázorňující závislost pozorované veličiny (ceny) v čase $t$ na ceně v předchozích 4 časech $t-1$, $t-2$, $t-3$ a $t-4$. Pozor na stejná měřítka os. Interpretujte grafy, navíc s pomocí ACF a PACF. 

In [ ]:
plt.figure(figsize=(10, 6))
for i in range(4):
    plt.subplot(1,4,i+1)
    plt.scatter(data[***], data[***])
    plt.title('Lag: {0}'.format(i+1))
    plt.axis('image')

NaFITujte mautoregresní model `AR` z balíčku `statsmodels.tsa.ar_model` na data. Je potřeba vhodně zvolit argument `maxlag`, jinak dostanete model výrazně komplikovanější. K tomu se lze propracovat kromě ACF/PACF rovněž pomocí metody `AR.select_order()`, jejíž nápovědu nalezneme rovněž níže. K dispozici máme několik kritérií IC - _information criterion_, vždy hledáme model s menší hodnotou kritéria. Oblíbená jsou:

- [AIC - Akaikeho informační kritérium](https://en.wikipedia.org/wiki/Akaike_information_criterion) - měří kvalitu FITu pomocí vztahu $AIC = 2p - 2\ln(L)$, kde $p$ je počet parametrů a $L$ je likelihood, věrohodnost fitovaných dat. Je asymptoticky ekvivalentní k leave-one-out křížové validaci lineárních regresních modelů.
- [BIC - Bayesovské informační kritérium](https://en.wikipedia.org/wiki/Bayesian_information_criterion) - funguje podobně, akorát podle vztahu $BIC = \ln(n)k - 2\ln(L)$, kde $n$ počet dat.

In [ ]:
#help(AR)

In [ ]:
model = AR(data)
order = model.select_order(maxlag=***, ic=***, trend=*** , method='cmle')
print('Řád AR modelu: ', order)

Nyní budeme FITovat :-)
Do proměnné `beta` uložte odhady (parametry) z FITu a koukněte na konfidenční intervaly na standardní hladině spolehlivosti 95%.

In [ ]:
result = model.fit(maxlag=***)
beta = result.***
print('Odhad vektoru regresních koeficientů: ', beta)
print('Konfidenční intervaly pro parametry (v řádcích):\n', result.conf_int())

Zajímá nás obvykle i odhad variance šumu. Uložte jej do proměnné `sigma2` a vytiskněte.

In [ ]:
sigma2 = result.***
print(f'Odhad variance šumu: {sigma2:.3f}')

Mělo by nás zajímat, zda je proces stacionární. To zjistíme z kořenů charakteristického polynomu položeného rovno nule:

$$
1 - \beta_1 z - \beta_2 z^2 - \ldots - \beta_p z^p  = 0,
$$

což v našem případě vede na jednoduchou rovnici

$$
1 - \beta_1 z = 0,
$$

s řešením

$$
z = \frac{1}{\beta_1}.
$$

Statsmodels toto spočte za nás:

In [ ]:
print('Kořen charakteristického polynomu: ', result.***)

Že proces není stacionární jsme v tomto jednoduchém případě poznali už z $|\beta_1|>1$. U složitějších modelů už by to takto přímočaře nešlo.

Zkusíme i pár predikcí budoucích hodnot, jejichž počet si předdefinujeme v proměnné `npreds`. K predikování můžeme použít jak metodu `result.predict()`, tak predikci pomocí skalárního součinu $[1, X_{t-1}]^\intercal \beta$.

Napočítejte 100 kroků predikce:

In [ ]:
npreds = 100
***

Vykreslíme data následovaná predikcemi.

In [ ]:
plt.plot(np.arange(ndat), data[:ndat], 'b')
plt.plot(np.arange(ndat, ndat+npreds), ***, 'r')

V `result.resid` najdeme rezidua regrese na datech, pomocí kterých jsme počítali odhady. Velmi užitečné je kouknout na jejich rozdělení, resp. histogram. Čekali bychom rezidua centrovaná v počátku, ideálně i symetrická.

In [ ]:
plt.hist(result.resid, bins=15)
plt.show()

**Mnohé z výše uvedených analýz lze zobrazit metodou `summary` :)**

In [ ]:
result.summary()